In [1]:
from pathlib import Path
import sys

from peewee import (
    SqliteDatabase,
)

sys.path.append(str(Path("../src").resolve().as_posix()))
from utils.database import (
    Organism,
    Sequence,
    TMInfo,
    Annotation,
)

db_path = Path("../data/tmvis.db")
db = SqliteDatabase(db_path.resolve().as_posix())
db.bind([Organism, Sequence, TMInfo, Annotation], bind_backrefs=False, bind_refs=False)

In [2]:
from peewee import OperationalError


def get_existing_indexes(model):
    """Fetch existing indexes for a given model."""
    table_name = model._meta.table_name
    existing_indexes = set()
    cursor = db.execute_sql(f'PRAGMA index_list("{table_name}");')
    for row in cursor.fetchall():
        existing_indexes.add(row[1])
    return existing_indexes


def create_missing_indexes(model):
    """Create indexes defined in the model but missing in the database."""
    existing_indexes = get_existing_indexes(model)
    missing_indexes = []

    for index in model._meta.indexes:
        index_fields, is_unique = index
        index_name = (
            "_".join(index_fields) if isinstance(index_fields, tuple) else index_fields
        )
        if index_name not in existing_indexes:
            missing_indexes.append((index_fields, is_unique))

    for index_fields, is_unique in missing_indexes:
        if isinstance(index_fields, str):
            index_fields = [index_fields]
        index_name = "_".join(index_fields)
        fields = ", ".join([f'"{field}"' for field in index_fields])
        unique = "UNIQUE" if is_unique else ""
        sql = f'CREATE {unique} INDEX IF NOT EXISTS "{index_name}" ON "{model._meta.table_name}" ({fields});'
        try:
            db.execute_sql(sql)
            print(f"Created index: {index_name}")
        except OperationalError as e:
            print(f"Error creating index {index_name}: {e}")


def ensure_indexes():
    models = [Organism, Sequence, TMInfo, Annotation]
    for model in models:
        create_missing_indexes(model)


with db:
    ensure_indexes()

Created index: super_kingdom_clade
Created index: has_alpha_helix_has_beta_strand_has_signal


In [ ]:
with db:
    db.execute_sql("REINDEX")